In [14]:
from typing import List, Tuple, Union, Set, Dict

In [15]:
from enum import Enum

class Operation(Enum):
    ADD = "+"
    SUBTRACT = "-"
    MULTIPLY = "*"
    DIVIDE = "/"
    POWER = "^"
    CONCAT = "><"

    __str__ = lambda self: self.value

CUMMUTATIVE = {Operation.ADD, Operation.MULTIPLY}

In [16]:
def phrase_order(phrase: List[Union[str, Operation]]) -> int:
    if isinstance(phrase, int):
        return phrase
    return min(
        value if isinstance(value, int) else float("inf") for value in phrase
    )

def get_operation(phrase: List[Union[int, Operation]]) -> List[Operation]:
    try:
        return phrase[0]
    except TypeError:
        return None

def order_commutative_phrases(
    phrase: List[Union[int, Operation]],
    operation_to_order: Operation
) -> List[Union[int, Operation]]:
    operation = phrase[0]
    left, right = phrase[1], phrase[2]
    left_operation, right_operation = get_operation(left), get_operation(right)

    if operation == operation_to_order:
        if all([left_operation == operation_to_order, right_operation == operation_to_order]):
            subphrases = [left[1], left[2], right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
        elif left_operation == operation_to_order:
            subphrases = [left[1], left[2], right]
            subphrases = sorted(subphrases, key=phrase_order)
        elif right_operation == operation_to_order:
            subphrases = [left, right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
        else:
            subphrases = [left, right]
            subphrases = sorted(subphrases, key=phrase_order)

        phrase = tuple([operation_to_order, subphrases[0], subphrases[1]])
        for subphrase in subphrases[2:]:
            phrase = tuple([operation_to_order, phrase, subphrase])

    return phrase

def invert_phrase(
    phrase: List[Union[int, Operation]],
    primary_operation: Operation,
    secondary_operation: Operation
) -> List[Union[int, Operation]]:
    operation, left, right = phrase[0], phrase[1], phrase[2]
    left_operation, right_operation = get_operation(left), get_operation(right)

    if operation == primary_operation:
        if all([left_operation == secondary_operation, right_operation == secondary_operation]):
            pass
        elif left_operation == secondary_operation:
            phrase = tuple([secondary_operation, tuple([primary_operation, left[1], right]), left[2]])
        elif right_operation == secondary_operation:
            phrase = tuple([secondary_operation, tuple([primary_operation, left, right[1]]), right[2]])

    return phrase

def invert_multiple_division(phrase: List[Union[int, Operation]]) -> List[Union[int, Operation]]:
    operation, left, right = phrase[0], phrase[1], phrase[2]
    right_operation = get_operation(right)

    if operation == Operation.DIVIDE and right_operation == Operation.DIVIDE:
        r_num, r_den = right[1], right[2]
        phrase = tuple([Operation.MULTIPLY, left, tuple([Operation.DIVIDE, r_den, r_num])])

    return phrase


def invert_subtraction(phrase: List[Union[int, Operation]]) -> List[Union[int, Operation]]:
    operation, left, right = phrase[0], phrase[1], phrase[2]

    def invert_subtraction_recursive(phrase: List[Union[int, Operation]]) -> Tuple[bool, list]:
        if isinstance(phrase, int):
            return False, phrase
        
        operation, left, right = phrase[0], phrase[1], phrase[2]
        
        if operation == Operation.SUBTRACT:
            return True, tuple([Operation.SUBTRACT, right, left])
        elif operation in [Operation.DIVIDE, Operation.MULTIPLY]:
            inverted_right = invert_subtraction_recursive(right)
            return inverted_right[0], tuple([operation, left, inverted_right[1]])
        else:
            return False, phrase

    # If there is another subtraction separated by multiplication or division, invert it's terms
    # and change the main operation to addition
    if operation == Operation.SUBTRACT:
        inverted_right = invert_subtraction_recursive(right)
        if inverted_right[0]:
            phrase = tuple([Operation.ADD, left, inverted_right[1]])
    return phrase

def canonicalize_phrase(phrase: List[Union[int, Operation]]) -> List[Union[int, Operation]]:
    if isinstance(phrase, int) or len(phrase) == 1:
        return phrase

    operation = phrase[0]

    if operation == Operation.MULTIPLY:
        phrase = invert_phrase(phrase, Operation.MULTIPLY, Operation.DIVIDE)
        phrase = order_commutative_phrases(phrase, Operation.MULTIPLY)

    elif operation == Operation.ADD:
        phrase = invert_phrase(phrase, Operation.ADD, Operation.SUBTRACT)
        phrase = order_commutative_phrases(phrase, Operation.ADD)

    elif operation == Operation.DIVIDE:
        phrase = invert_multiple_division(phrase)

    elif operation == Operation.SUBTRACT:
        phrase = invert_subtraction(phrase)

    operation, left, right = phrase[0], phrase[1], phrase[2]
    left, right = canonicalize_phrase(left), canonicalize_phrase(right)
    phrase = tuple([operation, left, right])
    
    if operation in CUMMUTATIVE and phrase_order(phrase[1]) > phrase_order(phrase[2]):
        phrase = tuple([operation, phrase[2], phrase[1]])
            
    return phrase

def generate_phrase_combinations(
    fingerprint_dictionary_one: Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]],
    fingerprint_dictionary_two: Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]],
    operations: List[Operation]
) -> Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]]:
    resulting_dictionary = {}

    for fingerprint_one, phrases_one in fingerprint_dictionary_one.items():
        for fingerprint_two, phrases_two in fingerprint_dictionary_two.items():
            fingerprint_one, fingerprint_two = set(fingerprint_one), set(fingerprint_two)
            if fingerprint_one & fingerprint_two:
                continue

            new_fingerprint = fingerprint_one | fingerprint_two
            new_fingerprint = tuple(sorted(new_fingerprint))
            num_digits = len(new_fingerprint)
            new_phrases = set()

            for phrase_one in phrases_one:
                for phrase_two in phrases_two:
                    for operation in operations:
                        valid_phrases = [
                            tuple([operation, phrase_one, phrase_two]),
                            tuple([operation, phrase_two, phrase_one])
                        ]

                        for valid_phrase in valid_phrases:
                            for _ in range(num_digits - 1):
                                valid_phrase = canonicalize_phrase(valid_phrase)
                            new_phrases.add(valid_phrase)

            if new_fingerprint in resulting_dictionary:
                resulting_dictionary[new_fingerprint] |= new_phrases
            else:
                resulting_dictionary[new_fingerprint] = new_phrases

    return resulting_dictionary

def generate_all_phrases(num_numbers: int, operations: List[str]) -> List[List[Union[int, Operation]]]:
    phrase_bank = {
        1: {
            tuple([i]): set([i]) for i in range(num_numbers)
        },
    }

    for current_phrase_length in range(1, num_numbers):
        for other_phrase_length in range(1, current_phrase_length + 1):
            if current_phrase_length + other_phrase_length > num_numbers:
                continue

            phrase_combinations = generate_phrase_combinations(
                phrase_bank[current_phrase_length],
                phrase_bank[other_phrase_length],
                operations
            )

            length = current_phrase_length + other_phrase_length
            if length in phrase_bank:
                for fingerprint, phrases in phrase_combinations.items():
                    if fingerprint in phrase_bank[length]:
                        phrase_bank[length][fingerprint] |= phrases
                    else:
                        phrase_bank[length][fingerprint] = phrases
            else:
                phrase_bank[length] = phrase_combinations

    return list(phrase_bank[num_numbers].values())[0]

In [17]:
def evaluate_permutation(
    permutation: List[Union[int, Operation]],
    numbers: List[int]
) -> int:
    if isinstance(permutation, int):
        return numbers[permutation]
    
    operation, left, right = permutation[0], permutation[1], permutation[2]
    left, right = evaluate_permutation(left, numbers), evaluate_permutation(right, numbers)
    if any([left is None, right is None]):
        return None

    if operation == Operation.ADD:
        result = left + right
    elif operation == Operation.SUBTRACT:
        result = left - right
    elif operation == Operation.MULTIPLY:
        result = left * right
    elif operation == Operation.DIVIDE:
        if right == 0:
            result = None
        else:
            result = left / right
            if not result.is_integer():
                result = None
            else:
                result = int(result)
    elif operation == Operation.POWER:
        if left <= 0 and right < 0:
            result = None
        else:
            result = left ** right
    elif operation == Operation.CONCAT:
        shift = 10 ** len(str(right))
        result = (left * shift) + right
    
    if not result is None and result > 100_000:
        return None
    
    return result
    

In [18]:
def pretty_print_expression(phrase: List[Union[int, Operation]], numbers: List[int]) -> str:
    if isinstance(phrase, int):
        return str(numbers[phrase])
    
    operation, left, right = phrase[0], phrase[1], phrase[2]
    left, right = pretty_print_expression(left, numbers), pretty_print_expression(right, numbers)
    
    if operation == Operation.ADD:
        return f"({left} + {right})"
    elif operation == Operation.SUBTRACT:
        return f"({left} - {right})"
    elif operation == Operation.MULTIPLY:
        return f"({left} * {right})"
    elif operation == Operation.DIVIDE:
        return f"({left} / {right})"
    elif operation == Operation.POWER:
        return f"({left} ^ {right})"
    elif operation == Operation.CONCAT:
        return f"({left} >< {right})"

In [19]:
def seed_permutation(permutation: tuple, numbers: List[int]):
    if isinstance(permutation, int):
        return numbers[permutation]
    
    operation, left, right = permutation[0], permutation[1], permutation[2]
    left, right = seed_permutation(left, numbers), seed_permutation(right, numbers)
    return tuple([operation, left, right])


def evaluate_all_permutations(
    phrases: List[Union[int, Operation]],
    numbers: List[int]
) -> Set[int]:
    results = {}
    expressions = {}
    for permutation in phrases:
        tuple_permutation = seed_permutation(permutation, numbers)
        if tuple_permutation in results:
            continue
        result = evaluate_permutation(permutation, numbers)
        if result is not None:
            results[tuple_permutation] = result
            expressions[tuple_permutation] = pretty_print_expression(permutation, numbers)
    
    results = list(results.values())
    expressions = list(expressions.values())
    return results, expressions

In [20]:
phrases = generate_all_phrases(
    4,
    [
        Operation.ADD,
        Operation.SUBTRACT,
        Operation.MULTIPLY,
        Operation.DIVIDE,
        Operation.POWER,
        Operation.CONCAT
    ]
)
len(phrases)

12372

In [21]:
numbers = [5, 3, 2, 3]
target = 24

numbers.sort()
results, expressions = evaluate_all_permutations(phrases, numbers)
num_equal_target = sum([result == target for result in results])
print(f"{num_equal_target} / {len(results)}")
print(num_equal_target / len(results))

for result, expression in zip(results, expressions):
    if result == target:
        print(expression)

15 / 3263
0.004596996628869139
(2 >< (5 - (3 / 3)))
(((2 + 5) * 3) + 3)
((5 ^ 2) - (3 / 3))
((3 >< 2) - (3 + 5))
((3 ^ 2) + (3 * 5))
(2 >< ((3 * 3) - 5))
(((3 >< 2) - 3) - 5)
(2 * ((3 * 5) - 3))
((3 ^ (5 - 2)) - 3)
(3 + ((2 + 5) * 3))
((2 >< 5) - (3 / 3))
((2 >< (3 * 3)) - 5)
(((5 - 2) ^ 3) - 3)
((2 + (3 ^ 3)) - 5)
(((3 >< 2) - 5) - 3)


In [22]:
all_groups_of_four_digits = []
for a in range(9):
    for b in range(9):
        for c in range(9):
            for d in range(9):
                digits = [a + 1, b + 1, c + 1, d + 1]
                digits.sort()
                digits = tuple(digits)
                all_groups_of_four_digits.append(digits)

all_groups_of_four_digits = list(set(all_groups_of_four_digits))
print(len(all_groups_of_four_digits))

495


In [23]:
results = []

for target in range(1, 64):
    sum_equal_target = 0
    total_possible = 0
    for digits in all_groups_of_four_digits:
        results, _ = evaluate_all_permutations(phrases, digits)
        num_equal_target = sum([result == target for result in results])
        sum_equal_target += num_equal_target
        if num_equal_target > 0:
            total_possible += 1

    results.append(sum_equal_target / total_possible)
    output = f"Target: {target}\nAvg Ways per Digit Group: {sum_equal_target / total_possible}\nProportion Possible: {total_possible / len(all_groups_of_four_digits)}\n"
    print(output, end="\r")
print(results)

Target: 1
Avg Ways per Digit Group: 133.86464646464646
Proportion Possible: 1.0
Target: 2
Avg Ways per Digit Group: 34.75151515151515
Proportion Possible: 1.0
Target: 3
Avg Ways per Digit Group: 31.44877049180328
Proportion Possible: 0.9858585858585859
Target: 4
Avg Ways per Digit Group: 33.913223140495866
Proportion Possible: 0.9777777777777777
Target: 5
Avg Ways per Digit Group: 27.06860706860707
Proportion Possible: 0.9717171717171718
Target: 6
Avg Ways per Digit Group: 28.053061224489795
Proportion Possible: 0.98989898989899
Target: 7
Avg Ways per Digit Group: 25.280163599182004
Proportion Possible: 0.9878787878787879
Target: 8
Avg Ways per Digit Group: 28.882113821138212
Proportion Possible: 0.9939393939393939
Target: 9
Avg Ways per Digit Group: 25.94297352342159
Proportion Possible: 0.9919191919191919
Target: 10
Avg Ways per Digit Group: 15.890020366598778
Proportion Possible: 0.9919191919191919
Target: 11
Avg Ways per Digit Group: 17.06276150627615
Proportion Possible: 0.9656565